In [1]:
import pandas as pd
import os, sys
import numpy as np
from pathlib import Path

data_root = Path('/home/glen/works/sr_lstm/data')

basin_id = '01529950'

In [4]:
with open(data_root / f'{basin_id}.txt', 'r') as f:
    lines = f.readlines()
lines = [l for l in lines if 'USGS' == l[0:4]]
with open(data_root / f'{basin_id}_discharge.txt', 'w') as f:
    f.writelines(lines)
df = pd.read_csv(data_root / f'{basin_id}_discharge.txt', sep="\t", header=None)
df.columns = ['Source', 'ID', 'Datetime', 'Time zone', 'discharge [ft^3/s]', 'Data status']
df['Datetime'] = pd.to_datetime(df['Datetime'])
df.set_index('Datetime', inplace=True)
df = df.iloc[:,:].resample('D').mean()
idx = pd.date_range('2000-01-01', '2018-01-01')
df = df.reindex(idx, fill_value=np.nan)
df.index.name = 'Date'
discharge = df['discharge [ft^3/s]']
discharge = discharge * 0.028317    # ft^3/s to m^3/s
df['Value'] = discharge
df.drop(columns=['discharge [ft^3/s]'], inplace=True)
df['ID'] = basin_id
df['PARAM'] = 1
df.reset_index(inplace=True)
df.loc[-1] = ['Date', 'ID', 'Value', 'PARAM']
df.index = df.index + 1
df = df.sort_index()
df = df[['ID', 'PARAM', 'Date', 'Value']]
os.remove(data_root / f'{basin_id}_discharge.txt')
os.makedirs(data_root / 'non_grip_gl' / f'{basin_id}')
df.to_csv(data_root / 'non_grip_gl' / f'{basin_id}' / f'{basin_id}_discharge.csv', index=False)

In [5]:
df

,ID,PARAM,Date,Value
0,ID,PARAM,Date,Value
1,01529950,1,2000-01-01 00:00:00,21.847132
2,01529950,1,2000-01-02 00:00:00,21.136941
3,01529950,1,2000-01-03 00:00:00,26.094682
4,01529950,1,2000-01-04 00:00:00,123.314872
...,...,...,...,...
6572,01529950,1,2017-12-28 00:00:00,16.455717
6573,01529950,1,2017-12-29 00:00:00,16.860414
6574,01529950,1,2017-12-30 00:00:00,16.734167
6575,01529950,1,2017-12-31 00:00:00,17.681607
